# Minigrad Autodiff Engine

This notebook contains a minimal implementation of an automatic differentiation engine that supports reverse-mode autodiff on N-dimensional tensors:
- Tensor: core data structure and computation graph
- Broadcasting & reductions: broadcasting in forward pass and unbroadcasting in backward pass
- Matmul: matrix multiplication and its vector-Jacobian products (VJPs)
- NN module: neural network library with Linear layers, activations, parameter collection
- Experiments: gradient checking, training simple models on toy datasets

Note: this project is self contained and won't be shared with CNN / Transformer / Diffusion, those projects will use Pytorch autograd.
